In [2]:
"""
Variable Scope and Binding

Non local variables:

Python 3 added a new keyword called nonlocal. The nonlocal keyword adds a scope override to the inner scope.  

"""
#If you try running this code, you will receive an UnboundLocalError because the num variable is referenced before
#it is assigned in the innermost function.

def counter():
    num = 0
    def incrementer():
        num += 1   #UnboundLocalError
        return num
    return incrementer()
c = counter()
print(c)

UnboundLocalError: local variable 'num' referenced before assignment

In [3]:
"""
Add the nonlocal to the above example.
"""

def counter():
    num = 0
    def incrementer():
        """
        Basically nonlocal will allow you to assign to variables in an outer scope, but not a global scope. 
        So you can't use nonlocal in our counter function because then it would try to assign to a global scope. 
        Give it a try and you will quickly get a SyntaxError. Instead you must use nonlocal in a nested function.
        """
        nonlocal num
        num += 1   
        return num
    return incrementer()
c = counter()
print(c)

1


In [6]:
num = 1
def counter():
    nonlocal num 
    def incrementer():
        num += 1   
        return num
    return incrementer()
c = counter()
print(c)

SyntaxError: no binding for nonlocal 'num' found (<ipython-input-6-f946bcb32c6a>, line 3)

In [7]:
"""
Global Variables

In Python, variables inside functions are considered local if and only if they appear in the left side of an assignment
statement, or some other binding occurrence; otherwise such a binding is looked up in enclosing functions, up to
the global scope. This is true even if the assignment statement is never executed.

"""

x = 'Hi'
def read_x():
    print(x) # x is just referenced, therefore assumed global

read_x() # prints Hi

def read_y():
    print(y) # here y is just referenced, therefore assumed global

read_y() # NameError: global name 'y' is not defined


Hi


NameError: name 'y' is not defined

In [8]:
def read_y():
    y = 'Hey' # y appears in an assignment, therefore it's local
    print(y) # will find the local y

read_y() # prints Hey

def read_x_local_fail():
    if False:
        x = 'Hey' # x appears in an assignment, therefore it's local
    print(x) # will look for the _local_ z, which is not assigned, and will not be found

read_x_local_fail() # UnboundLocalError: local variable 'x' referenced before assignment


Hey


UnboundLocalError: local variable 'x' referenced before assignment

In [9]:
#Normally, an assignment inside a scope will shadow any outer variables of the same name:
x = 'Hi'
def change_local_x():
    x = 'Bye'
    print(x)
change_local_x() # prints Bye
print(x) # prints Hi

Bye
Hi


In [10]:
"""
Declaring a name global means that, for the rest of the scope, any assignments to the name will happen at the
module's top level

The global keyword means that assignments will happen at the module's top level, not at the program's top level.
Other modules will still need the usual dotted access to variables within the module.
To summarize: in order to know whether a variable x is local to a function, you should read the entire function:
1. if you've found global x, then x is a global variable
2. If you've found nonlocal x, then x belongs to an enclosing function, and is neither local nor global
3. If you've found x = 5 or for x in range(3) or some other binding, then x is a local variable
4. Otherwise x belongs to some enclosing scope (function scope, global scope, or builtins)
"""

x = 'Hi'
def change_global_x():
    global x
    x = 'Bye'
    print(x)

change_global_x() # prints Bye
print(x) # prints Bye

Bye
Bye


In [11]:
"""
Local Variables

If a name is bound inside a function, it is by default accessible only within the function

"""
def foo():
    a = 5
    print(a) # ok
print(a) # NameError: name 'a' is not defined


NameError: name 'a' is not defined

In [13]:
"""

Control flow constructs have no impact on the scope (with the exception of except), but accessing variable that was
not assigned yet is an error:

Note : Common binding operations are assignments, for loops, and augmented assignments such as a += 5
"""

def foo():
    if True:
        a = 5
    print(a) # ok
foo()    
b = 3
def bar():
    if False:
        b = 5
    print(b) # UnboundLocalError: local variable 'b' referenced before assignment
bar()


5


UnboundLocalError: local variable 'b' referenced before assignment

In [14]:
"""
The del command

This command has several related yet distinct forms.
Syntax: 
del v

If v is a variable, the command del v removes the variable from its scope.

"""
x = 5
print(x) # out: 5
del x
print(x) # NameError: name 'x' is not defined

"""

Note that del is a binding occurrence, which means that unless explicitly stated otherwise (using nonlocal
or global), del v will make v local to the current scope. If you intend to delete v in an outer scope, use
nonlocal v or global v in the same scope of the del v statement.
"""



5


NameError: name 'x' is not defined

In [15]:
"""
In all the following, the intention of a command is a default behavior but is not enforced by the language. A class
might be written in a way that invalidates this intention.
del v.name
This command triggers a call to v.__delattr__(name).
The intention is to make the attribute name unavailable. For example:

"""

class A:
    pass
a = A()
a.x = 7
print(a.x) # out: 7
del a.x
print(a.x) # error: AttributeError: 'A' object has no attribute 'x'



7


AttributeError: 'A' object has no attribute 'x'

In [16]:
"""
del v[item]
This command triggers a call to v.__delitem__(item).
The intention is that item will not belong in the mapping implemented by the object v. For example:

"""

x = {'a': 1, 'b': 2}
del x['a']
print(x) # out: {'b': 2}
print(x['a']) # error: KeyError: 'a'

{'b': 2}


KeyError: 'a'

In [17]:
"""
del v[a:b]
This actually calls v.__delslice__(a, b).
The intention is similar to the one described above, but with slices - ranges of items instead of a single item. For
example:
"""
x = [0, 1, 2, 3, 4]
del x[1:3]
print(x) # out: [0, 3, 4]

[0, 3, 4]


In [18]:
"""
Functions skip class scope when looking up names

Classes have a local scope during definition, but functions inside the class do not use that scope when looking up
names. Because lambdas are functions, and comprehensions are implemented using function scope, this can lead
to some surprising behavior.
"""

a = 'global'
class Fred:
    a = 'class' # class scope
    b = (a for i in range(10)) # function scope
    c = [a for i in range(10)] # function scope
    d = a # class scope
    e = lambda: a # function scope
    f = lambda a=a: a # default argument uses class scope
    
    @staticmethod # or @classmethod, or regular instance method
    def g(): # function scope
        return a

print(Fred.a) # class
print(next(Fred.b)) # global
print(Fred.c[0]) # class in Python 2, global in Python 3
print(Fred.d) # class
print(Fred.e()) # global
print(Fred.f()) # class
print(Fred.g()) # global

class
global
global
class
global
class
global


In [23]:
"""
From PEP 227:
Names in class scope are not accessible. Names are resolved in the innermost enclosing function scope.
If a class definition occurs in a chain of nested scopes, the resolution process skips class definitions.

From Python Documentation, naming and binding

The scope of names defined in a class block is limited to the class block; it does not extend to the code
blocks of methods – this includes comprehensions and generator expressions since they are
implemented using a function scope. This means that the following will fail

"""

class A:
    x = 4
    y = list(x + i for i in range(10))
v = A()
print(v.x)
print(v.y)

SyntaxError: invalid syntax (<ipython-input-23-54cc37dc76a3>, line 16)

In [24]:
"""
The above example fails, New way below example works in v3.0
"""
class A:
    x = 4
    y = (lambda x=x: [x+i for i in range(1)])()
v = A()
print(v.x)
print(v.y)   

4
[4]


In [28]:
"""
Local vs Global Scope

What are local and global scope?
All Python variables which are accessible at some point in code are either in local scope or in global scope.
The explanation is that local scope includes all variables defined in the current function and global scope includes
variables defined outside of the current function.

"""
foo = 1 # global
def func():
    bar = 2 # local
    print(foo) # prints variable foo from global scope
    print(bar) # prints variable bar from local scope
func()

1
2


In [30]:
"""
One can inspect which variables are in which scope. Built-in functions locals() and globals() return the whole
scopes as dictionaries.

"""
foo = 1
def func():
    bar = 2
    print(globals().keys(),"\n") # prints all variable names in global scope
    print(locals().keys()) # prints all variable names in local scope

func()

dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_1', '_i2', 'counter', '_i3', 'c', '_i4', '_i5', 'num', '_i6', '_i7', 'read_x', 'read_y', '_i8', 'read_x_local_fail', '_i9', 'change_local_x', '_i10', 'change_global_x', '_i11', 'foo', '_i12', 'b', 'bar', '_i13', '_i14', '_i15', 'A', 'a', '_i16', 'x', '_i17', '_i18', 'Fred', '_i19', '_i20', '_i21', '_i22', '_i23', '_i24', 'v', '_i25', 'func', '_i26', '_i27', '_i28', '_i29', '_i30']) 

dict_keys(['bar'])


In [31]:
"""
What happens with name clashes?

"""
foo = 1
def func():
    foo = 2 # creates a new variable foo in local scope, global foo is not affected
    print(foo) # prints 2
    # global variable foo still exists, unchanged:
    print(globals()['foo']) # prints 1
    print(locals()['foo']) # prints 2
func()

2
1
2


In [32]:
#To modify a global variable, use keyword global:
foo = 1
def func():
    global foo
    foo = 2 # this modifies the global foo, rather than creating a local variable
    print(foo)
func()
print(foo)

2
2


In [33]:
"""
The scope is defined for the whole body of the function!
What it means is that a variable will never be global for a half of the function and local afterwards, or vice-versa.
"""
foo = 1
def func():
    # This function has a local variable foo, because it is defined down below.
    # So, foo is local from this point. Global foo is hidden.
    
    print(foo) # raises UnboundLocalError, because local foo is not yet initialized
    foo = 7
    print(foo)
func()

UnboundLocalError: local variable 'foo' referenced before assignment

In [37]:
#Likewise, the opposite:
foo = 1
def func():
    # In this function, foo is a global variable from the beginning
    foo = 7 # global foo is modified
    
    print(foo) # 7
    print(globals()['foo']) # 7
    
    global foo # this could be anywhere within the function - Failed, before using it make it global
    print(foo) # 7
func()

SyntaxError: name 'foo' is used prior to global declaration (<ipython-input-37-60118c8e48ec>, line 13)

In [38]:
"""
Functions within functions
There may be many levels of functions nested within functions, but within any one function there is only one local
scope for that function and the global scope. There are no intermediate scopes.

"""

foo = 1
def f1():
    print("In Function : f1()...............................................")
    bar = 1
    
    def f2():
        print("In Function : f2()...............................................")
        baz = 2
        # here, foo is a global variable, baz is a local variable
        # bar is not in either scope
        print(locals().keys()) # ['baz']
        print('bar' in locals()) # False
        print('bar' in globals()) # False
        
    def f3():
        print("In Function : f3()...............................................")
        baz = 3
        print(bar) # bar from f1 is referenced so it enters local scope of f3 (closure)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals()) # False
    
    def f4():
        print("In Function : f4()...............................................")
        bar = 4 # a new local bar which hides bar from local scope of f1
        baz = 4
        print(bar)
        print(locals().keys()) # ['bar', 'baz']
        print('bar' in locals()) # True
        print('bar' in globals()) # False
    f2()
    f3()
    f4()
        
f1()

In Function : f1()...............................................
In Function : f2()...............................................
dict_keys(['baz'])
False
True
In Function : f3()...............................................
1
dict_keys(['baz', 'bar'])
True
True
In Function : f4()...............................................
4
dict_keys(['bar', 'baz'])
True
True


In [46]:
"""
global vs nonlocal (Python 3 only)
Both these keywords are used to gain write access to variables which are not local to the current functions.
The global keyword declares that a name should be treated as a global variable.

"""

foo = 0 # global foo
def f1():
    print("In Function : f1() ->")
    foo = 1 # a new foo local in f1
    def f2():
        print("In Function : f1() -> f2() ->")
        foo = 2 # a new foo local in f2
        print("Initially, In f2(), foo : ", foo )
        def f3():
            print("In Function : f1() -> f2() -> f3() ->")
            foo = 3 # a new foo local in f3
            print("foo : ", foo) # 3
            foo = 30 # modifies local foo in f3 only
        def f4():
            print("In Function : f1() -> f2() ->f4() ->")
            global foo
            print("foo : ", foo) # 0
            foo = 100 # modifies global foo
        f3()
        f4()
        print("Now In f2(), foo : ", foo )
    print("In f1(), foo : ", foo )
    f2()
print("Initially, Global Scope, foo :  ", foo)
f1()
print("Global Scope, foo :  ", foo)

Initially, Global Scope, foo :   0
In Function : f1() ->
In f1(), foo :  1
In Function : f1() -> f2() ->
Initially, In f2(), foo :  2
In Function : f1() -> f2() -> f3() ->
foo :  3
In Function : f1() -> f2() ->f4() ->
foo :  0
Now In f2(), foo :  2
Global Scope, foo :   100


In [47]:
"""
nonlocal (see Nonlocal Variables ), available in Python 3, takes a local variable from an
enclosing scope into the local scope of current function.
From the Python documentation on nonlocal:
The nonlocal statement causes the listed identifiers to refer to previously bound variables in the nearest
enclosing scope excluding globals.
Python 3.x Version ≥ 3.0
"""
def f1():
    def f2():
        foo = 2 # a new foo local in f2
        def f3():
            nonlocal foo # foo from f2, which is the nearest enclosing scope
            print(foo) # 2
            foo = 20 # modifies foo from f2!
        f3()
    f2()
f1()



2


In [48]:
#Binding Occurrence
x = 5
x += 7
for x in iterable: pass
"""
Each of the above statements is a binding occurrence - x become bound to the object denoted by 5. If this statement
appears inside a function, then x will be function-local by default. See the "Syntax" section for a list of binding
statements.
"""

NameError: name 'iterable' is not defined